## APTOS 2019 Blindness Detection
---
In this Kernel, we will design a Machine learning model,which will help in identifing the eyes disease.  As this is a imaged based problem, we will use Deep Learning for model design.

Diabetic retinopathy affects blood vessels in the light-sensitive tissue called the retina that lines the back of the eye. It is the most common cause of vision loss among people with diabetes and the leading cause of vision impairment and blindness among working-age adults. It don't have any earaly symtoms. As of now, Retena photography is a way to detect the stage of Blindness. Automating it with ml, will help a lot in health domain. 

---------------------------------------
1. [Import Required Libraries](#1)
1. [Loading Data ](#2)
1. [Data Visualization](#3)
1. [Train and Test dataset](#4)
1. [Data Pre-Processing](#6)
1. [Image Data Generator](#7)
1. [Model Architecture Design](#8)
1. [Keras Callback Funcations](#9)
1. [Transfer Learning](#10)
1. [Validation Accuracy & Loss](#11)
1. [Validation Accuracy](#12)
1. [Test-Time Augmentation](#13)
1. [Visualization Test Result](#14)
------------------------------------
- Design CNN from Scratch
- Use pre-train model for Blindness Detection
 
 Stages Of Diabetic Retinopathy
- NO DR
- Mild
- Moderate 
- Servere
- Proliferative DR

<a id="1"></a> 
# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import PIL
import gc
import psutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import set_random_seed
from tqdm import tqdm
from math import ceil
import math
import sys
import gc

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

from keras.activations import softmax
from keras.activations import elu
from keras.activations import relu
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from tqdm import tqdm

gc.enable()

print(os.listdir("../input/"))

<a id="2"></a>
#### Exploratory Data Analysis
- Loading Data 
- Data Disribution
- Data Visualization

Setup all the param, which we will use in model


In [ ]:
SEED = 7
np.random.seed(SEED)
set_random_seed(SEED)
dir_path = "../input/aptos2019-blindness-detection/"
IMG_DIM = 224  # 224 399 #
BATCH_SIZE = 12
CHANNEL_SIZE = 3
NUM_EPOCHS = 60
size=224
TRAIN_DIR = 'train_images'
TEST_DIR = 'test_images'
FREEZE_LAYERS = 2  # freeze the first this many layers for training
CLASSS = {0: "No DR", 1: "Mild", 2: "Moderate", 3: "Severe", 4: "Proliferative DR"}

<a id="2"></a>
## Loading Data 

In [ ]:
df_train = pd.read_csv(os.path.join(dir_path, "train.csv"))
df_test = pd.read_csv(os.path.join(dir_path, "test.csv"))
NUM_CLASSES = df_train['diagnosis'].nunique()

In [ ]:
print("Training set has {} samples and {} classes.".format(df_train.shape[0], df_train.shape[1]))
print("Testing set has {} samples and {} classes.".format(df_test.shape[0], df_test.shape[1]))

In [ ]:
def get_cropped_image(image):
    img = cv2.blur(image,(2,2))
    slice1Copy = np.uint8(img)
    canny = cv2.Canny(slice1Copy, 0, 50)
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    cropped_img = img[y1:y2, x1:x2]
    cropped_img = cv2.resize(cropped_img, size)
    return cropped_img

In [ ]:
def load_image(path):
    img=cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img = cv2.resize(cv2.imread(path), size)
    img = get_cropped_image(img)
    return img

<a id="6"></a>
#### Split DataSet

In [ ]:
#labels = df_train["diagnosis"].values.tolist()
#labels = keras.utils.to_categorical(labels)

In [ ]:
x_trai, x_val, y_train, y_val = train_test_split(df_train.id_code,df_train["diagnosis"], test_size=0.15,
                                                    random_state=SEED,stratify=df_train["diagnosis"])

In [ ]:
y_train = pd.get_dummies(y_train).values
y_valid = pd.get_dummies(y_val).values

<a id="7"></a>
# Image Data Generator
In this section willl use Keras ImageDataGenerator class for generating data for Keras model. It is used for data generation, increasing the data size. with the help of ImageDataGenerator we will do image "augment" via a number of random transformations, so that our model would never see twice the exact same picture. 

Training Deep Learning model can perform better with more data, and augementation technique can create variations of data that can increase the ababiliy of fit model to gene



In [ ]:
# Creating the imageDatagenerator Instance 
datagenerator=ImageDataGenerator(#rescale=1./255,
#                                       validation_split=0.15, 
                                         horizontal_flip=True,
                                         vertical_flip=True, 
                                         rotation_range=40, 
                                         zoom_range=0.2, 
                                         shear_range=0.1,
                                        fill_mode='nearest')

In [ ]:
imgPath = f"../input/aptos2019-blindness-detection/train_images/cd54d022e37d.png"
# Loading image
img = load_img(imgPath)
data = img_to_array(img)
samples =np.expand_dims(data, 0)
i=5
it=datagenerator.flow(samples , batch_size=1)
for i in range(5):
    plt.subplot(230 + 1 + i)
    batch = it.next()
    image = batch[0].astype('uint8')
    plt.imshow(image)
plt.show()

In [ ]:
from tqdm import tqdm_notebook
train_dir = "../input/aptos2019-blindness-detection/train_images/"
training_paths = [train_dir + str(x) + str(".png") for x in df_train["id_code"]]
images = np.empty((len(df_train), 256,256,3), dtype = np.uint8)
for i, path in tqdm_notebook(enumerate(training_paths)):
    images[i,:,:,:] = load_img(path)

In [ ]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                         vertical_flip=True, 
                                         rotation_range=60, 
                                         zoom_range=0.2, 
                                         shear_range=0.1,
                                        fill_mode='nearest')
# valid_datagen=image.ImageDataGenerator(rescale=1./255)

SINCE ABOVE PREPROCESS CODE ISNT WORKING WE WORK WITH PREPROCESSED IMGS FROM XCEPTION

In [ ]:
x_train = np.load("../input/preprocessed-aptos-images/x_train_224_2019.npy", mmap_mode=None, allow_pickle=False, fix_imports=True)

In [ ]:
x_valid=np.load("../input/preprocessed-aptos-images/x_valid_224_2019.npy", mmap_mode=None, allow_pickle=False, fix_imports=True)

In [ ]:
train_generator = train_datagen.flow(x_train,y_train,batch_size=12)
valid_generator = train_datagen.flow(x_valid,y_valid,batch_size=12
                                                    )
#del x_train
# # del x_test
#del y_train
# del y_test
gc.collect()
#  color_mode= "grayscale",


<a id="9"></a>
# Keras Callback Funcations
- Call Back functions Eraly Stoping and Learning Rate Reducing

In [ ]:
#eraly_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
# Reducing the Learning Rate if result is not improving. 
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)

In [ ]:
NUB_TRAIN_STEPS = train_generator.n // train_generator.batch_size
NUB_VALID_STEPS = valid_generator.n // valid_generator.batch_size

NUB_TRAIN_STEPS, NUB_VALID_STEPS

<a id="10"></a>
# Transfer Learning 

In [ ]:
from keras.layers import LeakyReLU
def create_resnet(img_dim, CHANNEL, n_class):
    input_tensor = Input(shape=(224, 224, CHANNEL))
    base_model = ResNet50(weights=None, include_top=False, input_tensor=input_tensor)
    base_model.load_weights('../input/resnet50weightsfile/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(2048)(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.3)(x)
    #x.add(LeakyReLU(alpha=0.1))
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.3)(x)
    #x.add(LeakyReLU(alpha=0.1))
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(n_class, activation='softmax', name="Output_Layer")(x)
    model_resnet = Model(input_tensor, output_layer)

    return model_resnet


model_resnet = create_resnet(IMG_DIM, CHANNEL_SIZE, NUM_CLASSES)

In [ ]:
# # Layers 
# for i, lay in enumerate(model_resnet.layers):
#     print(i,lay.name)
# Training All Layers

for layers in model_resnet.layers:
    layers.trainable = True


In [ ]:
lr = 1e-3
optimizer = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True) # Adam(lr=lr, decay=0.01) 
model_resnet.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
# model.summary()
gc.collect()


In [ ]:
history = model_resnet.fit_generator(generator=train_generator,
                                     steps_per_epoch=NUB_TRAIN_STEPS,
                                     validation_data=valid_generator,
                                     validation_steps=NUB_VALID_STEPS,
                                     epochs=15, 
                                     callbacks=[reduce_lr],
                                     verbose=2)
model_resnet.save('leakyresnet2702.h5')
gc.collect()



In [ ]:
history.history.keys()

In [ ]:
accu = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(accu, label="Accuracy")
plt.plot(val_acc)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['Acc', 'val_acc'])
plt.plot(np.argmax(history.history["val_acc"]), np.max(history.history["val_acc"]), marker="x", color="r",
         label="best model")
plt.show()



In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r",
         label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();


<a id="12"></a>
## Validation Accuracy

In [ ]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
(eval_loss, eval_accuracy) = tqdm(
    model_resnet.evaluate_generator(generator=valid_generator, steps=NUB_VALID_STEPS, pickle_safe=False))
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))


In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2, horizontal_flip=True)

test_generator = test_datagen.flow_from_dataframe(dataframe=df_test,
                                                  directory="../input/aptos2019-blindness-detection/test_images/",
                                                  x_col="id_code",
                                                  target_size=(IMG_DIM, IMG_DIM),
                                                  batch_size=1,
                                                  shuffle=False,
                                                  class_mode=None,
                                                  seed=SEED)
# del df_test
print(df_test.shape[0])
# del train_datagen
# del traabsin_generator
gc.collect()


Kapkaha

<a id="13"></a>
# Test-Time Augmentation
In the below section, we are doning TTA imporving the prediction accuracy. It will transform image and predict 

In [ ]:
tta_steps = 5
preds_tta = []
for i in tqdm(range(tta_steps)):
    test_generator.reset()
    preds = model_resnet.predict_generator(generator=test_generator, steps=ceil(df_test.shape[0]))
    #     print('Before ', preds.shape)
    preds_tta.append(preds)
#     print(i,  len(preds_tta))


In [ ]:
final_pred = np.mean(preds_tta, axis=0)
predicted_class_indices = np.argmax(final_pred, axis=1)
len(predicted_class_indices)


In [ ]:
# test_generator.filenames.apply(lambda x: x[-4])
results = pd.DataFrame({"id_code": test_generator.filenames, "diagnosis": predicted_class_indices})
results.id_code = results.id_code.apply(lambda x: x[:-4])  # results.head()
results.to_csv("submission.csv", index=False)


  <a id="14"></a>
 # Visualization Test Result
- this section will visualize the predicted classes of test data.

In [ ]:
results['diagnosis'].value_counts().plot(kind='bar')
plt.title('Test Samples Per Class')


References:

1. https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1
1. https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
1. https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/
1. https://jkjung-avt.github.io/keras-image-cropping/
1. https://www.kaggle.com/aleksandradeis/aptos2019-blindness-detection-eda